```
StringSpelledByGappedPatterns(GappedPatterns, k, d)
    FirstPatterns ← the sequence of initial k-mers from GappedPatterns
    SecondPatterns ← the sequence of terminal k-mers from GappedPatterns
    PrefixString ← StringSpelledByPatterns(FirstPatterns, k)
    SuffixString ← StringSpelledByPatterns(SecondPatterns, k)
    for i = k + d + 1 to |PrefixString|
        if the i-th symbol in PrefixString does not equal the (i - k - d)-th symbol in SuffixString
            return "there is no string spelled by the gapped patterns"
    return PrefixString concatenated with the last k + d symbols of SuffixString
```

In [1]:
from collections import defaultdict, deque

def build_de_bruijn_graph(gapped_patterns):
    """Constrói o grafo de de Bruijn para os padrões emparelhados."""
    graph = defaultdict(list)
    for pair in gapped_patterns:
        prefix = (pair[0][:-1], pair[1][:-1])
        suffix = (pair[0][1:], pair[1][1:])
        graph[prefix].append(suffix)
    return graph

def find_eulerian_path(graph):
    """Encontra um caminho Euleriano no grafo."""
    in_degree = defaultdict(int)
    out_degree = defaultdict(int)
    for node, neighbors in graph.items():
        out_degree[node] += len(neighbors)
        for neighbor in neighbors:
            in_degree[neighbor] += 1
    start_node = None
    for node in set(in_degree.keys()).union(out_degree.keys()):
        if out_degree[node] > in_degree[node]:
            start_node = node
            break
    stack = [start_node]
    path = []
    while stack:
        node = stack[-1]
        if graph[node]:
            stack.append(graph[node].pop())
        else:
            path.append(stack.pop())
    return path[::-1]

def StringSpelledByPatterns(patterns, k):
    """Reconstrói uma string a partir de uma lista de k-mers."""
    result = patterns[0]
    for i in range(1, len(patterns)):
        result += patterns[i][-1]
    return result

def StringSpelledByGappedPatterns(GappedPatterns, k, d):
    """Reconstrói uma string a partir de padrões com gap."""
    # Construir o grafo de de Bruijn
    graph = build_de_bruijn_graph(GappedPatterns)
    # Encontrar o caminho Euleriano
    path = find_eulerian_path(graph)
    # Separar os padrões iniciais e finais do caminho
    FirstPatterns = [node[0] for node in path]
    SecondPatterns = [node[1] for node in path]
    # Reconstituir as strings prefix e suffix
    PrefixString = StringSpelledByPatterns(FirstPatterns, k)
    SuffixString = StringSpelledByPatterns(SecondPatterns, k)
    # Verificar consistência entre PrefixString e SuffixString
    for i in range(k + d, len(PrefixString)):
        if PrefixString[i] != SuffixString[i - k - d]:
            return "There is no string spelled by the gapped patterns"
    # Concatenar o PrefixString com os últimos k + d símbolos de SuffixString
    return PrefixString + SuffixString[-(k + d):]

def parse_gapped_patterns(input_string):
    """Converte uma string de padrões emparelhados no formato 'GAGA|TTGA' para uma lista de tuplas."""
    pairs = input_string.strip().split()  # Dividir pelos espaços
    gapped_patterns = [tuple(pair.split('|'))
    for pair in pairs]  # Dividir pelo separador '|'
    return gapped_patterns

with open("dataset_30208_4.txt", "r") as file_1:
    linhas_1 = file_1.readlines()
    Text = linhas_1[1].strip()

k = 50
d = 200

GappedPatterns = parse_gapped_patterns(Text)
print(StringSpelledByGappedPatterns(GappedPatterns, k, d))

AGCCTTAAGTGAAGGGCAGAACATATGACCTAAGACGCAGAAAGGGCAGAACATATGACCTAAGACGCAGAACGGGTGCAGGAAGGCCGTGACGGGTGCAGGAAGGCCGTGAACCCATGCTCCAAGGGCAGAACATATGACCTAAGACGCAGAACGGGTGCAGGAAGGCCGTGGTCAATCCCAAGGGCAGAACATATGACCTAAGACGCAGAACGGGTGCAGGAAGGCCGTGGACGTTTGCTAACGGCATACGCTGGCAGTTTAAGGGCAGAACATATGACCTAAGACGCAGAACGGGTGCAGGAAGGCCGTGTGTCCGTTGTGCTGCCGTCTTCTTCAGGAAAGGGCAGAACATATGACCTAAGACGCAGAACGGGTGCAGGAAGGCCGTGCCTCTTAGAAGATCCGCCGAACGACTGGTTCATTCGACATGCTGCGTAAAGTCAAGGGCAGAACATATGACCTAAGACGCAGAACGGGTGCAGGAAGGCCGTGGTCTGTTCCCTTACATGGAATCTCCCTCTGGCCCCGCCCGTGTGAAGGGCAGAACATATGACCTAAGACGCAGAACGGGTGCAGGAAGGCCGTGCGTCGAAATTGAACGAAATCTCCAACCATTCTAGCAACTGCTGCCTAGTAATCGCTCATCTCCGAGTATGCCCCTGGGGCGGCGATTTTCGTTAAGGGCAGAACATATGACAAGGGCAGAACATATGACCTAAGACGCAGAACGGGTGCAGGAAGGCCGTGCTAAGACGCAGAACGGGTGCAGGAAGGCCGTGGATGGATTCCTCGGTTGTTTTACTGCTAGGAGAACGAAAGGGCAGAACATATGACCTAAGACGCAGAACGGGTGCAGGAAGGCCGTGCGGGGGGTGTTGCTAGAAAAAGGGCAGAACATATGACCTAAGACGCAGAACGGGTGCAGGAAGGCCGTGAGGCTTTTTTACCATTTTAACACAAACTGTTCGTACAAAGACCATTGACAGTCCTCCCTAAG